Objective for this lab is to:

1. Use Tools (Push notification tool, custom made tools to record user contact details and quesstions that LLM cant answer and send them as push notifications)
2. Learn to deploy gradio app on huggingface spaces 

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os
import json
import requests
from pypdf import PdfReader
import gradio as gr

In [25]:
load_dotenv(override=True)
openai = OpenAI()

In [4]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [ ]:
# Function to send a push notification using Pushover
def push(message):
    print(f"Push: {message}")
    payload = {"user":pushover_user, "token":pushover_token, "message":message}
    requests.post(pushover_url, data=payload)

In [6]:
push("HEY!")

Push: HEY!


In [7]:
# These two would be manually coded tools that we would provide to the agent.

def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes: {notes}")
    return {"recorded":"ok"}


def record_unknown_question(question):
    push(f"Recording {question} asked that I could not answer")
    return {"recorded":"ok"}

In [12]:
record_user_details_json = {
    "name": "record_user_details",
    "description":"Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of the user"
            }, 
            "name": {
                "type": "string",
                "description": "The name of the user, if provided"
            },
            "notes": {
                "type": "string",
                "description": "Any additional notes provided by the user"
            }
        },      
        "required": ["email"],
        "additionalProperties": False
    
}}




record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [13]:
tools = [{"type": "function", "function": record_user_details_json},
         {"type": "function", "function": record_unknown_question_json}]    

In [19]:
# Function below would be able to run the tools provided as lists (Its the IF statement version)

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}, flush = True")


        # This is the AI i.e. IF statement that would run the tools

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        result.append({"role":"tool","content":json.dumps(result), "tool_call_id":tool_call.id})
    return results

In [23]:
# Thsi is how you can handle those if statements in a more generic way using globals
globals()["record_unknown_question"]("This a test question")

Push: Recording This a test question asked that I could not answer


{'recorded': 'ok'}

In [24]:
# Better way to handle tool calls and remove IF statements

def handle_tool_calls_v2(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}, flush = True")
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role":"tool","content":json.dumps(result), "tool_call_id":tool_call.id})

    return results

In [30]:
reader = PdfReader("me/Linkedin.pdf")

Linkdin = ""
for page in reader.pages:
    text = page.extract_text()
    
    if text:
        Linkdin += text



with open("me/summary.txt", "r", encoding='utf-8') as f:
    summary = f.read()


name = "Jayraj Parmar"

In [34]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{Linkdin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [39]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

I am not going to deploy the app because I'll deploy something better on my website, Gradio feels very basic and not sure if you can beautify the chatbot structure and window

## And now for deployment

This code is in `app.py`

We will deploy to HuggingFace Spaces. Thank you student Robert M for improving these instructions.

Before you start: remember to update the files in the "me" directory - your LinkedIn profile and summary.txt - so that it talks about you!  
Also check that there's no README file within the 1_foundations directory. If there is one, please delete it. The deploy process creates a new README file in this directory for you.

1. Visit https://huggingface.co and set up an account  
2. From the Avatar menu on the top right, choose Access Tokens. Choose "Create New Token". Give it WRITE permissions.
3. Take this token and add it to your .env file: `HF_TOKEN=hf_xxx` and see note below if this token doesn't seem to get picked up during deployment  
4. From the 1_foundations folder, enter: `uv run gradio deploy` and if for some reason this still wants you to enter your HF token, then interrupt it with ctrl+c and run this instead: `uv run dotenv -f ../.env run -- uv run gradio deploy` which forces your keys to all be set as environment variables   
5. Follow its instructions: name it "career_conversation", specify app.py, choose cpu-basic as the hardware, say Yes to needing to supply secrets, provide your openai api key, your pushover user and token, and say "no" to github actions.  

#### Extra note about the HuggingFace token

A couple of students have mentioned the HuggingFace doesn't detect their token, even though it's in the .env file. Here are things to try:   
1. Restart Cursor   
2. Rerun load_dotenv(override=True) and use a new terminal (the + button on the top right of the Terminal)   
3. In the Terminal, run this before the gradio deploy: `$env:HF_TOKEN = "hf_XXXX"`  
Thank you James and Martins for these tips.  

#### More about these secrets:

If you're confused by what's going on with these secrets: it just wants you to enter the key name and value for each of your secrets -- so you would enter:  
`OPENAI_API_KEY`  
Followed by:  
`sk-proj-...`  

And if you don't want to set secrets this way, or something goes wrong with it, it's no problem - you can change your secrets later:  
1. Log in to HuggingFace website  
2. Go to your profile screen via the Avatar menu on the top right  
3. Select the Space you deployed  
4. Click on the Settings wheel on the top right  
5. You can scroll down to change your secrets, delete the space, etc.

#### And now you should be deployed!

Here is mine: https://huggingface.co/spaces/ed-donner/Career_Conversation

I just got a push notification that a student asked me how they can become President of their country 😂😂

For more information on deployment:

https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces

To delete your Space in the future:  
1. Log in to HuggingFace
2. From the Avatar menu, select your profile
3. Click on the Space itself and select the settings wheel on the top right
4. Scroll to the Delete section at the bottom
5. ALSO: delete the README file that Gradio may have created inside this 1_foundations folder (otherwise it won't ask you the questions the next time you do a gradio deploy)
